In [1]:
import os
from pathlib import Path
from multiprocessing import Pool
from xml.etree.ElementTree import iterparse, tostring

In [2]:
data_dir = Path('../../../data/stackoverflow/raw')

In [5]:
tags = ['game-engine',
        'game-physics',
        'game-development',
        'gameobject',
        '2d-games',
        'unreal-engine4',
        'unreal-blueprint',
        'unreal-development-kit',
        'unrealscript',
        'unity3d',
        'unity5',
        'unity5.3',
        'unity3d-mecanim',
        'unity3d-terrain',
        'unityscript',
        'unity3d-2dtools',
        'unity3d-unet',
        'unity-webgl',
        'unity2d',
        'unity-editor',
        'unity3d-editor',
        'unity-networking',
        'unity3d-gui',
        'unity-ui',
        'unity3d-5']

def make_questions_xml(tags):
    def split_xml(tag):
        print(tag)
        os.system(
            f'grep -F "&lt;{tag}&gt;" {data_dir}/Posts.xml > {data_dir}/tag_questions/{tag}.xml')
    def tag_pool(tags):
        with Pool(8) as p:
            p.map(split_xml, tags)

    def merge_files():
        os.system(f'echo "<posts>" > {data_dir}/gamedev_questions.xml')
        os.system(
            f'cat {data_dir}/tag_questions/* >> {data_dir}/gamedev_questions.xml')
        os.system(f'echo "</posts>" >> {data_dir}/gamedev_questions.xml')
    
    tag_pool(tags)  # takes 21 mins
    merge_files()
    
def get_ids(name):
    gamedev_ids = set()
    for _, node in iterparse(data_dir / f'gamedev_{name}.xml', events=("end",)):
        if node.tag == "row":
            gamedev_ids.add(node.attrib.get('Id'))
    return gamedev_ids

def to_xml(file_name, xml_list):
    os.system(f'echo "<posts>" > {data_dir}/{file_name}.xml')
    
    with open(f'{data_dir}/{file_name}.xml', 'a') as f:
        for xml in xml_list:
            f.write(xml.decode('utf-8'))
    
    os.system(f'echo "</posts>" >> {data_dir}/{file_name}.xml')
    
def make_answers_xml():
    ids = get_ids('questions')
    
    # 28 mins
    answers = []
    for _, node in iterparse(data_dir / 'Posts.xml', events=("end",)):
        if node.tag == "row" and node.attrib.get('PostTypeId') == '2' and node.attrib.get('ParentId') in ids:
            answers.append(tostring(node))
        node.clear()
    
    to_xml('gamedev_answers', answers)
    
def make_comments_xml():
    ids = get_ids('questions') | get_ids('answers')
    
    # 9 mins
    comments = []
    for _, node in iterparse(data_dir / 'Comments.xml', events=("end",)):
        if node.tag == "row" and node.attrib.get('PostId') in ids:
            comments.append(tostring(node))
        node.clear()
    
    to_xml('gamedev_comments', comments)

In [6]:
#make_questions_xml(tags)
#make_answers_xml()
make_comments_xml()